# LSTM sin embeddings

Modelo básico con embeddings entrenados acá. Un baseline :-)

In [1]:
import keras

Using TensorFlow backend.


In [2]:
%ls ../data/dev_es/

dev_es.tsv  train_es.tsv


In [13]:
import pandas as pd
df_train = pd.read_table("../data/dev_es/train_es.tsv", index_col="id")
df_dev = pd.read_table("../data/dev_es/dev_es.tsv", index_col="id")

text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]

In [17]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 100000

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_dev = tokenizer.texts_to_sequences(text_dev)

max_length = 30

X_train = pad_sequences(X_train, max_length)
X_dev = pad_sequences(X_dev, max_length)

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, BI
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

embedding_vector_length = 32

model = Sequential()
model.add(Embedding(num_words, embedding_vector_length, input_length=max_length))
model.add(LSTM(100))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 30, 32)            3200000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               12928     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 3,266,257
Trainable params: 3,266,257
Non-trainable params: 0
_________________________________________________________________
None
Train on 4469 samples, validate on 500 samples
Epoch 1/10
4469/4469 [==============================] - 19s 4ms/step - loss: 0.6144 - acc: 0.6583 - val_loss: 0.5623 - val_acc: 0.7180
Epoch 2/10
4469/4469 [==============================] - 16s 4ms/step - loss: 

In [25]:
loss, accuracy = model.evaluate(X_dev, y_dev)

print("Loss function: {}".format(loss))
print("Accuracy: {}%".format(accuracy*100))

500/500 [==============================] - 0s 624us/step
Loss function: 1.9181310005187988
Accuracy: 72.6%


## Bidirectional LSTM

In [27]:
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

embedding_vector_length = 32

model = Sequential()
model.add(Embedding(num_words, embedding_vector_length, input_length=max_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 30, 32)            3200000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               106400    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               25728     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 3,332,257
Trainable params: 3,332,257
Non-trainable params: 0
_________________________________________________________________
None
Train on 4469 samples, validate on 500 samples
Epoch 1/10
4469/4469 [==============================] - 25s 6ms/step - loss: 0.6109 - acc: 0.6594 - val_loss: 0.5417 - val_acc: 0.7080
Epoch 2/10
4469/4469 [==============================] - 20s 5ms/step - loss: 